In [18]:
from matplotlib import pyplot
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10


(X_train, y_train), (X_test, y_test) = cifar10.load_data()
tf.keras.utils.to_categorical(
    y, num_classes=None, dtype='float32'
)

NameError: name 'y' is not defined

In [11]:
print(X_train.shape)

(50000, 32, 32, 3)


In [12]:

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Flatten, Activation

model = Sequential([
    
    Convolution2D(filters = 5, kernel_size = (3,3), activation = 'relu', input_shape=(32,32,3)),
    MaxPooling2D(pool_size=(2,2)),
    BatchNormalization(),
    #do a drop out here if interested
    
    Convolution2D(filters = 25, kernel_size = (3,3), activation = 'relu'),
    MaxPooling2D(pool_size=(2,2)),
    BatchNormalization(),
    #do a drop out here if interested
    
    Flatten(),
    Dense(10),
    Activation('softmax')
])

In [13]:
model.compile('adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']
             )

In [16]:
history = model.fit(X_train, y_train, epochs = 10, validation_split = 0.1)

Epoch 1/10


ValueError: in user code:

    /Users/eiffelwong1/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Users/eiffelwong1/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/eiffelwong1/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/eiffelwong1/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/eiffelwong1/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/eiffelw